In [2]:
import osmnx as ox
import networkx as nx
import geopandas as gpd

### Prepare network
This notebook loads OpenStreetMap data and uses osmnx to extract a routable network graph. Several filters are applied and the network is cleaned before writing it out.

In [1]:
# Manage inputs and outputs
osm_path = "../../results/network/network.osm"
output_path = "../../results/network/network.default.graphml"

excluded_area_path = None #  "../../resources/spatial/zfe.gpkg"
excluded_tags = None
crs = "EPSG:2154"

if "snakemake" in locals():
    osm_path = snakemake.input["osm"]

    if "excluded_area" in snakemake.input.keys():
        excluded_area_path = snakemake.input["excluded_area"]

    params = snakemake.params[0] if len(snakemake.params) == 1 and len(snakemake.params.keys()) == 0 else snakemake.params

    if "excluded_tags" in params:
        excluded_tags = params["excluded_tags"]

    if "crs" in params:
        crs = params["crs"]

    output_path = snakemake.output[0]

In [3]:
# Load the data
graph = ox.graph_from_xml(osm_path, bidirectional = True)

In [4]:
# Filter out areas if requested
if excluded_area_path is not None:
    df_excluded = gpd.read_file(excluded_area_path).to_crs(crs)
    print("Nodes before filtering:", len(graph.nodes))

    df_nodes = ox.utils_graph.graph_to_gdfs(graph, nodes = True, edges = False).reset_index()[[
        "osmid", "geometry"]].to_crs(crs)
    
    df_nodes = gpd.sjoin(df_nodes, df_excluded, predicate = "within")
    graph.remove_nodes_from(df_nodes["osmid"].values)

    print("Nodes after filtering:", len(graph.nodes))

Nodes before filtering: 151286
Nodes after filtering: 120526


In [5]:
# Filter out road tags if requested
if excluded_tags is not None:
    print("Edges before filtering:", len(graph.edges))

    excluded_edges = set()
    for u, v, data in graph.edges(data = True):
        if data["highway"] in excluded_tags:
            excluded_edges.add((u, v))

    graph.remove_edges_from(excluded_edges)
    
    print("Edges after filtering:", len(graph.edges))

In [6]:
## Network clean-up

# Self-loop edges
self_loop_edges = list(nx.selfloop_edges(graph))
graph.remove_edges_from(self_loop_edges)

# Isolated nodes
graph = ox.utils_graph.remove_isolated_nodes(graph)

# Find strongly connected component
graph = ox.utils_graph.get_largest_component(graph, strongly = True)

# Speeds and travel times
graph = ox.add_edge_speeds(graph)
graph = ox.add_edge_travel_times(graph)

In [8]:
# Output
ox.io.save_graphml(graph, output_path)